In [2]:
import pandas as pd


In [3]:
df = pd.read_json("processed_extracted_metadata.json")

In [4]:
df

,id,Instance of,Title,Genre,Duration,Publication date,Country of origin,MPA Film Rating,Assessment,Award Received,...,Characters,Series,Disney+ movie ID,Voice actor,Num Seasons,Num Episodes,Sunrise,Hulu ID,Sunset,Disney+ series ID
0,Q595,Q11424,The Intouchables,"[Q157443, Q645928, Q130232, Q52162262]",112,"[2011-09-23T00:00:00, {'value': '2012-01-05T00...",Q142,Q18665344,"[{'value': 'Q4165246', 'qualifiers': {'assessm...","[{'value': 'Q2354778', 'qualifiers': {'point i...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Q2345,Q11424,12 Angry Men,"[Q130232, Q3072039, Q121432339]",95,"[{'value': '1957-04-10T00:00:00', 'qualifiers'...",Q30,NaN,"[{'value': 'Q4165246', 'qualifiers': {'assessm...","[{'value': 'Q154590', 'qualifiers': {'point in...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Q3997,Q11424,Nalai Manithan,"[Q200092, Q188473, Q1054574]",NaN,"[1989-01-14T00:00:00, 1989-01-01T00:00:00]",Q668,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Q7216,Q11424,Pirates II: Stagnetti's Revenge,"[Q185529, Q2096633]",138,2008-09-27T00:00:00,Q30,NaN,"{'value': 'Q4165246', 'qualifiers': {'assessme...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Q11618,Q11424,Eraserhead,"[Q130232, Q102260466, Q2973181, Q157394]",89,"[1977-01-01T00:00:00, {'value': '1979-09-07T00...",Q30,NaN,"{'value': 'Q4165246', 'qualifiers': {'criterio...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370681,Q131549933,Q11424,"The Randall Scandal: Love, Loathing, and Vande...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1zFYV4qEUmPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370682,Q131549935,Q11424,The Roof,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4DAAAWKdea14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370683,Q131549957,Q11424,We Live Here: The Midwest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2muuvINZcRYJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370684,Q131550157,Q11424,I Want To Live (2015 film),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['id', 'Instance of', 'Title', 'Genre', 'Duration', 'Publication date',
       'Country of origin', 'MPA Film Rating', 'Assessment', 'Award Received',
       'Nominated for', 'Box office', 'Production budget',
       'Production company', 'Distribution format', 'Narrative location',
       'Cast member', 'Director', 'Screenwriter', 'Composer',
       'Director of photography', 'Film editor', 'Official Website',
       'Original language of work', 'Netflix ID', 'Based on', 'Creator',
       'Executive Producer', 'Characters', 'Series', 'Disney+ movie ID',
       'Voice actor', 'Num Seasons', 'Num Episodes', 'Sunrise', 'Hulu ID',
       'Sunset', 'Disney+ series ID'],
      dtype='object')

In [6]:
initial = df[['id', 'Instance of', 'Title', 'Duration', 'Netflix ID', "Disney+ movie ID", "Disney+ series ID", 'Hulu ID']]

In [7]:
initial.loc[initial['Instance of'] == 'Q11424', 'Instance of'] = 'movie'

C:\Users\krdan\AppData\Local\Temp\ipykernel_6532\1940172358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial.loc[initial['Instance of'] == 'Q11424', 'Instance of'] = 'movie'


In [8]:
initial.loc[initial['Instance of'] =='Q15416', 'Instance of'] = 'tv'

C:\Users\krdan\AppData\Local\Temp\ipykernel_6532\2345560530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial.loc[initial['Instance of'] =='Q15416', 'Instance of'] = 'tv'


In [9]:
initial.loc[~initial['Instance of'].isin(['movie', 'tv']), 'Instance of'] = initial.loc[~initial['Instance of'].isin(['movie', 'tv']), 'Instance of'].apply(lambda x: 'movie' if 'Q11424' in x else x)
initial.loc[~initial['Instance of'].isin(['movie', 'tv']), 'Instance of'] = initial.loc[~initial['Instance of'].isin(['movie', 'tv']), 'Instance of'].apply(lambda x: 'tv' if 'Q15416' in x else x)

C:\Users\krdan\AppData\Local\Temp\ipykernel_6532\477627052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial.loc[~initial['Instance of'].isin(['movie', 'tv']), 'Instance of'] = initial.loc[~initial['Instance of'].isin(['movie', 'tv']), 'Instance of'].apply(lambda x: 'movie' if 'Q11424' in x else x)
C:\Users\krdan\AppData\Local\Temp\ipykernel_6532\477627052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial.loc[~initial['Instance of'].isin(['movie', 'tv']), 'Instance of'] = initial.loc[~initial['Instance of'].isin(['movie', 'tv']), 'Instance of'].apply(lambda x: 'tv' if 'Q15416' in x else x)


In [31]:
initial.loc[~initial['Instance of'].isin(['movie', 'tv'])].iloc[0]['Instance of']

{'value': 'Q11424',
 'qualifiers': {'follows': [{'entity-type': 'item',
    'numeric-id': 24953,
    'id': 'Q24953'}]}}

In [34]:
initial.loc[~initial['Instance of'].isin(['movie', 'tv'])].iloc[3]['Instance of']

[{'value': 'Q1555508',
  'qualifiers': {'publisher': [{'entity-type': 'item',
     'numeric-id': 1351173,
     'id': 'Q1351173'}]}},
 {'value': 'Q15416',
  'qualifiers': {'publisher': [{'entity-type': 'item',
     'numeric-id': 1881062,
     'id': 'Q1881062'}]}}]

In [ ]:
def clean_instance(value):
    """
    Select the best duration value from the input data.
    
    The function supports different data types:
      - If 'value' is a list of dicts (each with a 'value' and optional 'qualifiers'),
        it picks the candidate with the highest priority.
      - If 'value' is a list of plain numeric (or string convertible) values,
        it converts them and returns the minimum.
      - If 'value' is a numeric or string, it converts it appropriately.
    
    Returns:
      A rounded numeric duration (an int) or None if conversion fails.
    """
    if isinstance(value, str):
        if value == 'Q11424':
            return 'movie'
        elif value == 'Q15416':
            return 'tv'
        else:
            return value
    elif isinstance(value, dict):
        if value.get('value') == 'Q11424':
            return 'movie'
        elif value.get('value') == 'Q15416':
            return 'tv'
        else:
            return value
    # Case 1: 'value' is a list.
    if isinstance(value, list):
        # Check if the list elements are dictionaries (i.e. structured with qualifiers).
        if all(isinstance(item, dict) for item in value):
            for item in value:
                



    # Fallback: unrecognized type.
    else:
        return None

In [11]:
initial.head()

,id,Instance of,Title,Duration,Netflix ID,Disney+ movie ID,Disney+ series ID,Hulu ID
0,Q595,movie,The Intouchables,112,70232180,NaN,NaN,NaN
1,Q2345,movie,12 Angry Men,95,60004251,NaN,NaN,NaN
2,Q3997,movie,Nalai Manithan,NaN,NaN,NaN,NaN,NaN
3,Q7216,movie,Pirates II: Stagnetti's Revenge,138,70113743,NaN,NaN,NaN
4,Q11618,movie,Eraserhead,89,70034219,NaN,NaN,NaN


In [12]:
initial.describe()

,id,Instance of,Title,Duration,Netflix ID,Disney+ movie ID,Disney+ series ID,Hulu ID
count,370686,370686,370618,136126,23281,1342,24,82
unique,370686,82,250755,1960,23280,1342,24,82
top,Q131550813,movie,,90,81288436,2muuvINZcRYJ,2pLOiKfNP9Ma,555f6cd8-bfd0-43cd-9144-6ec61140ba44
freq,1,313179,88312,8372,2,1,1,1


In [13]:
def made_in_us(x):
    if isinstance(x, list):
        return 'Q30' in x
    elif isinstance(x, str):
        return x == 'Q30'

In [14]:
import numpy as np
import math

def is_non_scalar(value):
    return isinstance(value, (list, dict, set, tuple, np.ndarray))

def get_candidate_priority(candidate):
    """
    Determine the candidate's priority based on its qualifiers.
    Lower numbers indicate higher priority.
    
    Priority rules:
      1. If the candidate's 'applies to part' qualifier has id 'Q26225765', return priority 1.
      2. Else if the candidate's 'place of publication' qualifier equals ['Q30'], return priority 2.
      3. Otherwise, return priority 3.
    """
    qualifiers = candidate.get('qualifiers', {})
    # Highest priority: "applies to part" equals Q26225765
    applies_to = qualifiers.get('applies to part')
    if isinstance(applies_to, list) and applies_to:
        # Check the first (or any) candidate; adjust if multiple values need to be considered.
        if isinstance(applies_to[0], dict) and applies_to[0].get('id') == 'Q26225765':
            return 1
    # Next priority: "place of publication" equals ['Q30']
    pop = qualifiers.get('place of publication')
    if isinstance(pop, list) and pop == ['Q30']:
        return 2
    # Lower priority for any candidate that doesn’t match the above.
    return 3

def choose_duration(value):
    """
    Select the best duration value from the input data.
    
    The function supports different data types:
      - If 'value' is a list of dicts (each with a 'value' and optional 'qualifiers'),
        it picks the candidate with the highest priority.
      - If 'value' is a list of plain numeric (or string convertible) values,
        it converts them and returns the minimum.
      - If 'value' is a numeric or string, it converts it appropriately.
    
    Returns:
      A rounded numeric duration (an int) or None if conversion fails.
    """
    # Case 1: 'value' is a list.
    if isinstance(value, list):
        # Check if the list elements are dictionaries (i.e. structured with qualifiers).
        if all(isinstance(item, dict) for item in value):
            # Filter out candidates that have a 'value' key.
            candidates = [item for item in value if 'value' in item]
            if not candidates:
                return None
            # Assign a priority to each candidate.
            candidates_with_priority = [
                (get_candidate_priority(candidate), candidate) for candidate in candidates
            ]
            # Sort by priority (lowest number is highest priority).
            candidates_with_priority.sort(key=lambda x: x[0])
            chosen_candidate = candidates_with_priority[0][1]
            chosen_value = chosen_candidate.get('value')
            try:
                # Convert the chosen value to float and round it.
                return round(float(chosen_value))
            except (ValueError, TypeError):
                return None
        else:
            # Otherwise, assume the list contains plain numeric or string values.
            numeric_candidates = []
            for item in value:
                try:
                    numeric_candidates.append(float(item))
                except (ValueError, TypeError):
                    continue
            if numeric_candidates:
                # Here you might choose the minimum (or maximum) duration.
                return round(min(numeric_candidates))
            else:
                return None

    # Case 2: 'value' is a numeric type.
    elif isinstance(value, (int, float)):
        if isinstance(value, float) and math.isnan(value):
            return None
        try:
            return round(float(value))
        except (ValueError, TypeError):
            return None

    # Case 3: 'value' is a string.
    elif isinstance(value, str):
        try:
            return round(float(value))
        except ValueError:
            return None

    # Fallback: unrecognized type.
    else:
        return None
    
initial = initial.loc[(initial['Title'] != '')]
initial['Duration'] = initial['Duration'].apply(choose_duration)

In [15]:
# import numpy as np
# import math
# def is_non_scalar(value):
#     return isinstance(value, (list, dict, set, tuple, np.ndarray))

# def choose_duration(value):
#     if isinstance(value, list) and isinstance(value[0], dict):
#         for versions in value:
#             if isinstance(versions, dict):
#                 if versions.get('qualifiers').get('applies to part'):
#                     if versions.get('qualifiers').get('applies to part')[0]['id'] == 'Q26225765':
#                         return versions.get('value')
#                 if versions.get('qualifiers').get('place of publication'):
#                     if versions.get('qualifiers').get('place of publication') == ['Q30']:
#                         return versions.get('value')
#         return value[0]['value']
#     elif isinstance(value, list):
#         print(value)
#         for idx, versions in enumerate(value):
#             if isinstance(versions, dict):
#                 if versions.get('qualifiers').get('applies to part'):
#                     if versions.get('qualifiers').get('applies to part')[0]['id'] == 'Q26225765':
#                         return versions.get('value')
#                     else:
#                         value.pop(idx)
#                 elif versions.get('qualifiers').get('place of publication'):
#                     if versions.get('qualifiers').get('place of publication') == ['Q30']:
#                         return versions.get('value')
#                     else:
#                         value.pop(idx)
#                 else:
#                     value.pop(idx)
#         print(value)
#         return min(value)

#     elif isinstance(value, (int, float)):
#         if math.isnan(value):
#             return None
#         return round(value)
#     elif isinstance(value, str):
#         return round(float(value))

# initial = initial.loc[(initial['Title'] != '')]
# initial['Duration'] = initial['Duration'].apply(choose_duration)

In [16]:
initial

,id,Instance of,Title,Duration,Netflix ID,Disney+ movie ID,Disney+ series ID,Hulu ID
0,Q595,movie,The Intouchables,112.0,70232180,NaN,NaN,NaN
1,Q2345,movie,12 Angry Men,95.0,60004251,NaN,NaN,NaN
2,Q3997,movie,Nalai Manithan,NaN,NaN,NaN,NaN,NaN
3,Q7216,movie,Pirates II: Stagnetti's Revenge,138.0,70113743,NaN,NaN,NaN
4,Q11618,movie,Eraserhead,89.0,70034219,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
370681,Q131549933,movie,"The Randall Scandal: Love, Loathing, and Vande...",NaN,NaN,1zFYV4qEUmPL,NaN,NaN
370682,Q131549935,movie,The Roof,NaN,NaN,4DAAAWKdea14,NaN,NaN
370683,Q131549957,movie,We Live Here: The Midwest,NaN,NaN,2muuvINZcRYJ,NaN,NaN
370684,Q131550157,movie,I Want To Live (2015 film),NaN,NaN,NaN,NaN,NaN


In [17]:
def check_for_list_or_dict(row):
    if isinstance(row, (int, float)):
        return False
    for item in row:
        if isinstance(item, list) or isinstance(item, dict):
            return True
    return False

In [18]:
initial.loc[initial['Disney+ movie ID'].apply(check_for_list_or_dict)]

,id,Instance of,Title,Duration,Netflix ID,Disney+ movie ID,Disney+ series ID,Hulu ID
202,Q188439,movie,Tangled,100.0,70128681,"[3V3ALy4SHStq, {'value': '5mb1AVLjOo2k', 'qual...",NaN,NaN
314,Q246283,movie,Frozen,101.0,70279000,"[4uKGzAJi3ROz, {'value': '1JffnkWmY6mD', 'qual...",NaN,NaN
10603,Q5815826,movie,Coco,109.0,80189213,"[{'value': 'db9orsI5O4gC', 'qualifiers': {'lan...",NaN,NaN
66515,Q14847067,movie,Into the Woods,125.0,70305948,"[1YYWziMOrDdq, {'value': '4XhpQPGupcwC', 'qual...",NaN,NaN
93316,Q322328,movie,The Nightmare Before Christmas,73.0,806284,"[5GjwOj5Rkpz2, {'value': '5f9Y4Fphlz7f', 'qual...",NaN,NaN
100677,Q4120951,movie,Muppets Most Wanted,107.0,70294662,"[5P53FrvtFCEo, {'value': 'drvTprAamsAx', 'qual...",NaN,NaN
129059,Q84323848,movie,Hamilton,160.0,NaN,"[3uPmBHWlO6HJ, {'value': '2fGLm1Uttylg', 'qual...",NaN,NaN
256908,Q19946102,movie,Beauty and the Beast,129.0,80135073,"[227lND2lKmLZ, {'value': 'uGi98kFbxOIp', 'qual...",NaN,NaN
306114,Q24272881,movie,Mary Poppins Returns,130.0,NaN,"[5F6U4wl2xb7P, {'value': '1CqJzEniH6X7', 'qual...",NaN,NaN


In [19]:
initial.loc[initial['Disney+ series ID'].apply(check_for_list_or_dict)]

,id,Instance of,Title,Duration,Netflix ID,Disney+ movie ID,Disney+ series ID,Hulu ID


In [20]:
initial.loc[initial['Hulu ID'].apply(check_for_list_or_dict)]

,id,Instance of,Title,Duration,Netflix ID,Disney+ movie ID,Disney+ series ID,Hulu ID


In [21]:
#Specific Netflix IDs to fix:
problematic_netflix_ids = ('Q289127', 'Q21001674', 'Q320588', 'Q13897247', 'Q20495759')
problematic_netflix_ids = initial.loc[initial['Netflix ID'].apply(check_for_list_or_dict)]['id'].tolist()
problematic_disney_movie_ids = initial.loc[initial['Disney+ movie ID'].apply(check_for_list_or_dict)]['id'].tolist()
problematic_disney_series_ids = initial.loc[initial['Disney+ series ID'].apply(check_for_list_or_dict)]['id'].tolist()
problematic_hulu_ids = initial.loc[initial['Hulu ID'].apply(check_for_list_or_dict)]['id'].tolist()

In [22]:
def flatten_list(value):
    row = []
    if isinstance(value, list):
        for val in value:
            if isinstance(val, dict):
                row.append(val['value'])
            else:
                row.append(val)
        return row
    return value



In [23]:
initial.loc[initial['id'].isin(problematic_netflix_ids), 'Netflix ID'] = \
    initial.loc[initial['id'].isin(problematic_netflix_ids), 'Netflix ID'].apply(flatten_list)

initial.loc[initial['id'].isin(problematic_disney_movie_ids), 'Disney+ movie ID'] = \
    initial.loc[initial['id'].isin(problematic_disney_movie_ids), 'Disney+ movie ID'].apply(flatten_list)

In [24]:
initial.loc[initial['Instance of'] != 'Q11424'].apply(flatten_list)

,id,Instance of,Title,Duration,Netflix ID,Disney+ movie ID,Disney+ series ID,Hulu ID
0,Q595,movie,The Intouchables,112.0,70232180,NaN,NaN,NaN
1,Q2345,movie,12 Angry Men,95.0,60004251,NaN,NaN,NaN
2,Q3997,movie,Nalai Manithan,NaN,NaN,NaN,NaN,NaN
3,Q7216,movie,Pirates II: Stagnetti's Revenge,138.0,70113743,NaN,NaN,NaN
4,Q11618,movie,Eraserhead,89.0,70034219,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
370681,Q131549933,movie,"The Randall Scandal: Love, Loathing, and Vande...",NaN,NaN,1zFYV4qEUmPL,NaN,NaN
370682,Q131549935,movie,The Roof,NaN,NaN,4DAAAWKdea14,NaN,NaN
370683,Q131549957,movie,We Live Here: The Midwest,NaN,NaN,2muuvINZcRYJ,NaN,NaN
370684,Q131550157,movie,I Want To Live (2015 film),NaN,NaN,NaN,NaN,NaN


In [25]:
initial.loc[initial['id'].isin(problematic_disney_movie_ids)]

,id,Instance of,Title,Duration,Netflix ID,Disney+ movie ID,Disney+ series ID,Hulu ID
202,Q188439,movie,Tangled,100.0,70128681,"[3V3ALy4SHStq, 5mb1AVLjOo2k]",NaN,NaN
314,Q246283,movie,Frozen,101.0,70279000,"[4uKGzAJi3ROz, 1JffnkWmY6mD]",NaN,NaN
10603,Q5815826,movie,Coco,109.0,80189213,"[db9orsI5O4gC, SvWfRZO0Z3mF, 4M9wtfJ9CfQm]",NaN,NaN
66515,Q14847067,movie,Into the Woods,125.0,70305948,"[1YYWziMOrDdq, 4XhpQPGupcwC]",NaN,NaN
93316,Q322328,movie,The Nightmare Before Christmas,73.0,806284,"[5GjwOj5Rkpz2, 5f9Y4Fphlz7f]",NaN,NaN
100677,Q4120951,movie,Muppets Most Wanted,107.0,70294662,"[5P53FrvtFCEo, drvTprAamsAx]",NaN,NaN
129059,Q84323848,movie,Hamilton,160.0,NaN,"[3uPmBHWlO6HJ, 2fGLm1Uttylg]",NaN,NaN
256908,Q19946102,movie,Beauty and the Beast,129.0,80135073,"[227lND2lKmLZ, uGi98kFbxOIp]",NaN,NaN
306114,Q24272881,movie,Mary Poppins Returns,130.0,NaN,"[5F6U4wl2xb7P, 1CqJzEniH6X7]",NaN,NaN


In [26]:
df['Instance of'].apply(lambda x: x['value'] if isinstance(x, dict) else x)

0         Q11424
1         Q11424
2         Q11424
3         Q11424
4         Q11424
           ...  
370681    Q11424
370682    Q11424
370683    Q11424
370684    Q11424
370685    Q11424
Name: Instance of, Length: 370686, dtype: object

In [27]:
print(df[['id', 'Genre']].dropna().explode('Genre'))

                id      Genre
0             Q595    Q157443
0             Q595    Q645928
0             Q595    Q130232
0             Q595  Q52162262
1            Q2345    Q130232
...            ...        ...
370663  Q131481060    Q202866
370667  Q131523693     Q93204
370668  Q131531679  Q20442589
370668  Q131531679     Q24862
370685  Q131550813   Q2484376

[332364 rows x 2 columns]


In [28]:
# import sqlalchemy
# from urllib.parse import quote_plus

# DB_HOST = "35.236.65.215"
# DB_USER = "dannydhkim"
# DB_PASSWORD = "!@#$Mari03240306"
# DB_NAME = "popcorn_metadata"
# DB_PORT = "3306"

# encoded_password = quote_plus(DB_PASSWORD)
# # Connect to MySQL
# engine = sqlalchemy.create_engine(f"mysql+pymysql://{DB_USER}:{encoded_password}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
# df = df[['id', 'Genre']].dropna().explode('Genre')
# df = df.rename(columns={"id": "content_id", "Genre": "genre_id"})
# df.to_sql("content_genre", con=engine, if_exists="append", index=False)

# print("Data successfully written to MySQL!")


In [29]:
print(df['Title'].loc[df['Title'].duplicated()].sort_values().tail(15))

318839    None
319411    None
322674    None
323890    None
324286    None
341370    None
341449    None
341559    None
341820    None
341898    None
351932    None
352784    None
357358    None
363877    None
369109    None
Name: Title, dtype: object


In [30]:
print(df['Title'].loc[df['Title']==''])

223        
387        
448        
541        
790        
         ..
370654     
370655     
370660     
370662     
370670     
Name: Title, Length: 88312, dtype: object
